**BERT with Balanced and Enriched data**

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 9.8 MB/s eta 0:00:00


In [ ]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 16.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd 
from matplotlib import pyplot as plt

import torch 
from torch import nn, optim 
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from torchmetrics.functional import accuracy , f1_score

import pytorch_lightning as pl 
from pytorch_lightning.callbacks import TQDMProgressBar

import transformers 
from transformers import BertModel, BertConfig, AutoModel, BertTokenizerFast , RobertaModel , RobertaConfig , RobertaTokenizer

In [ ]:
pub_health_train = pd.read_csv('drive/MyDrive/Projects/pubhealth/data/clean/clean_train.tsv', sep='\t')
pub_health_test  = pd.read_csv('drive/MyDrive/Projects/pubhealth/data/clean/clean_test.tsv', sep='\t')

In [ ]:
pub_health_train.describe()

,label,claim,main_text,explanation,subjects,sources
count,9803,9803,9803,9803,9803,9803
unique,4,9803,9586,9659,5810,5062
top,true,"""The money the Clinton Foundation took from fr...",Doctors in India have been successful in treat...,Bird flu has been detected in turkeys in easte...,Health News,
freq,5077,1,12,2,1242,4550


In [ ]:
pub_health_train.head()

,label,claim,main_text,explanation,subjects,sources
0,false,"""The money the Clinton Foundation took from fr...","""Hillary Clinton is in the political crosshair...","""Gingrich said the Clinton Foundation """"took m...","Foreign Policy, PunditFact, Newt Gingrich,",https://www.wsj.com/articles/clinton-foundatio...
1,mixture,Annual Mammograms May Have More False-Positives,While the financial costs of screening mammogr...,This article reports on the results of a study...,"Screening,WebMD,women's health",
2,mixture,SBRT Offers Prostate Cancer Patients High Canc...,The news release quotes lead researcher Robert...,This news release describes five-year outcomes...,"Association/Society news release,Cancer",https://www.healthnewsreview.org/wp-content/up...
3,true,"Study: Vaccine for Breast, Ovarian Cancer Has ...","The story does discuss costs, but the framing ...","While the story does many things well, the ove...","Cancer,WebMD,women's health",http://clinicaltrials.gov/ct2/results?term=can...
4,true,Some appendicitis cases may not require ’emerg...,"""Although the story didn’t cite the cost of ap...",We really don’t understand why only a handful ...,,


In [ ]:
def tolist(col):
  '''column of string to list
  '''
  return col.to_list()

def avglist(strlist:list):
  '''get average of a list of strings
  '''
  return sum([len(s) for s in strlist])/len(strlist)

def average_columns_len(df,columns:list):
  ''' given a list of columns in data frame , the function should return s dict having as values an average of the lenght of sentence present in the columns which are keys
  '''
  res = dict()
  for col in columns:
    res[col]=round(avglist(tolist(df[col])),2)
  return res


In [ ]:
average_columns_len(pub_health_train,['claim','main_text','subjects','explanation','sources'])

{'claim': 85.69,
 'main_text': 4376.84,
 'subjects': 40.08,
 'explanation': 482.97,
 'sources': 390.47}

In [ ]:
# l = 0
# bigs = ''
# pbhealth = pub_health_train['sources'].to_list()
# for s in pbhealth:
#     s = str(s)
#     if len(s) > l:
#       l = len(s)
#       bigs = s


In [ ]:
pub_health_train['claim'][0]

'"The money the Clinton Foundation took from from foreign governments while Hillary Clinton was secretary of state ""is clearly illegal. … The Constitution says you can’t take this stuff."'

In [ ]:
pub_health_train['explanation'][0]

'"Gingrich said the Clinton Foundation ""took money from from foreign governments while (Hillary Clinton) was secretary of state. It is clearly illegal. … The Constitution says you can’t take this stuff."" A clause in the Constitution does prohibit U.S. officials such as former Secretary of State Hillary Clinton from receiving gifts, or emoluments, from foreign governments. But the gifts in this case were donations from foreign governments that went to the Clinton Foundation, not Hillary Clinton. She was not part of the foundation her husband founded while she was secretary of state. Does that violate the Constitution? Some libertarian-minded constitutional law experts say it very well could. Others are skeptical. What’s clear is there is room for ambiguity, and the donations are anything but ""clearly illegal."" The reality is this a hazy part of U.S. constitutional\xa0law. '

In [ ]:
pub_health_train['main_text'][0]

'"Hillary Clinton is in the political crosshairs as the author of a new book alleges improper financial ties between her public and personal life. At issue in conservative author Peter Schweizer’s forthcoming book Clinton Cash are donations from foreign governments to the Clinton Foundation during the four years she served as secretary of state. George Stephanopoulos used an interview with Schweizer on ABC This Week to point out what other nonpartisan journalists have found: There is no ""smoking gun"" showing that donations to the foundation influenced her foreign policy decisions. Still, former Republican House Speaker Newt Gingrich says the donations are ""clearly illegal"" under federal law. In his view, a donation by a foreign government to the Clinton Foundation while Clinton was secretary of state is the same as money sent directly to her, he said, even though she did not join the foundation’s board until she left her post. ""The Constitution of the United States says you cannot

In [ ]:
# pub_health_train = pub_health_train[['claim','explanation','subject','main_text', 'label']]
# pub_health_train = pub_health_train[['claim','explanation','subject','main_text', 'label']]
pub_health_train = pub_health_train[['subjects','main_text','label']]
pub_health_test = pub_health_test[['subjects','main_text','label']]

In [ ]:
pub_health_train['main_text_subj'] = 'Subject : ' + pub_health_train['subjects'] + '.' + 'Main : ' + pub_health_train['main_text']
pub_health_test['main_text_subj'] = 'Subject : ' + pub_health_test['subjects'] + '.' + 'Main : ' + pub_health_test['main_text']

In [ ]:
pub_health_train['label'] = pub_health_train['label'].map(
    {"true":0, "false":1, "unproven":2, "mixture":3}
)
pub_health_test['label'] = pub_health_test['label'].map(
    {"true":0, "false":1, "unproven":2, "mixture":3}
)

In [ ]:
pub_health_train.head()

,subjects,main_text,label,main_text_subj
0,"Health, Rhode Island, General News, Flu",The Rhode Island Department of Heath released ...,0,"Subject : Health, Rhode Island, General News, ..."
1,Health News,President Cyril Ramaphosa announced the 21-day...,0,Subject : Health News.Main : President Cyril R...
2,"melanoma,vitamins",Not applicable. The cost of Vitamin A is not i...,0,"Subject : melanoma,vitamins.Main : Not applica..."
3,Health News,In a joint statement after a five-day conferen...,0,Subject : Health News.Main : In a joint statem...
4,"Chicago, Illinois, Poisoning, AP Top News, Hyp...","In Illinois alone, hospitals reported more tha...",0,"Subject : Chicago, Illinois, Poisoning, AP Top..."


In [ ]:
class HealthClaimClassifier(pl.LightningModule):

    def __init__(self, max_seq_len=512, batch_size=32, learning_rate=1e-3):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        self.loss = nn.CrossEntropyLoss()

        self.pretrain_model = AutoModel.from_pretrained('bert-base-uncased', return_dict=False)
        self.pretrain_model.eval() 
        for param in self.pretrain_model.parameters():
            param.requires_grad = False
        
        self.new_layers = nn.Sequential(
            nn.Linear(768, 512), 
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(512, 4),
            nn.LogSoftmax(dim=1),
        )
    def prepare_data(self):
        tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', return_dict=False)
        
        tokens_train = tokenizer.batch_encode_plus(
            pub_health_train['main_text_subj'].tolist(),
            max_length=self.max_seq_len, 
            pad_to_max_length=True, 
            truncation=True, 
            return_token_type_ids=False
        )

        tokens_test = tokenizer.batch_encode_plus(            
            pub_health_test['main_text_subj'].tolist(),
            max_length=self.max_seq_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=False
        )

        self.train_seq = torch.tensor(tokens_train['input_ids'])
        self.train_mask = torch.tensor(tokens_train['attention_mask'])
        self.train_y = torch.tensor(pub_health_train['label'].tolist())

        self.test_seq = torch.tensor(tokens_test['input_ids'])
        self.test_mask = torch.tensor(tokens_test['attention_mask'])
        self.test_y = torch.tensor(pub_health_test['label'].tolist())

    def forward(self, seq, mask):
        _, output = self.pretrain_model(seq, attention_mask=mask)
        output = self.new_layers(output)
        return output
    
    def train_dataloader(self):
        train_dataset = TensorDataset(self.train_seq, self.train_mask, self.train_y)
        self.train_dataloader_obj = DataLoader(train_dataset, batch_size=self.batch_size)
        return self.train_dataloader_obj
    
    def test_dataloader(self):
        test_dataset = TensorDataset(self.test_seq, self.test_mask, self.test_y)
        self.test_dataloader_obj = DataLoader(test_dataset, batch_size=self.batch_size)
        return self.test_dataloader_obj
    
    def training_step(self, batch, batch_idx):
        seq, mask, targets = batch 

        outputs = self(seq, mask)
        preds = torch.argmax(outputs, dim=1)
        
        train_accuracy = accuracy(preds, targets,task="multiclass", num_classes=4)
        train_f1core = f1_score(preds, targets,task="multiclass", num_classes=4)

        loss = self.loss(outputs, targets)

        self.log('train_accuracy', train_accuracy, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_f1core', train_f1core, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_loss', loss, on_step=False, on_epoch=True)


        return {'loss': loss, 'train_accuracy': train_accuracy , 'train_f1core':train_f1core}
    
    def test_step(self, batch, batch_idx):
        seq, mask, targets = batch 

        outputs = self.forward(seq, mask)
        preds = torch.argmax(outputs, dim=1)

        test_accuracy = accuracy(preds, targets,task="multiclass", num_classes=4)
        test_f1core = f1_score(preds, targets,task="multiclass", num_classes=4)
        loss = self.loss(outputs, targets)
        return {'loss': loss, 'test_accuracy': test_accuracy,'test_f1core':test_f1core}

    def test_epoch_end(self, outputs):
        test_outs = []
        for test_out in outputs:
            out = test_out['test_accuracy']
            test_outs.append(out)
        total_test_accuracy = torch.stack(test_outs).mean()
        self.log('total_test_accuracy', total_test_accuracy, on_step=False, on_epoch=True)
        return total_test_accuracy
    
    def configure_optimizers(self):
        params = self.parameters()
        optimizer = torch.optim.Adam(params=params, lr=self.learning_rate)
        return optimizer

In [ ]:
# model = HealthClaimClassifier()
# trainer = pl.Trainer(fast_dev_run=True, accelerator='gpu', devices=1)


In [ ]:
model = HealthClaimClassifier()
bar = TQDMProgressBar()
# trainer = pl.Trainer(default_root_dir='BERT_transfer_learning_ckpts', max_epochs=10, accelerator='gpu', devices=1)
trainer = pl.Trainer(default_root_dir='BERT_transfer_learning_ckpts', max_epochs=10 , accelerator='gpu', devices=1)

histo = trainer.fit(model)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0

Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
# model = HealthClaimClassifier()
# bar = TQDMProgressBar()
# trainer = pl.Trainer(default_root_dir='BERT_transfer_learning_ckpts', max_epochs=10, accelerator='gpu', devices=1, callbacks=[bar])
# histo = trainer.fit(model)

In [ ]:
trainer.test()

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/checkpoint_connector.py:134: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at BERT_transfer_learning_ckpts/lightning_logs/version_1/checkpoints/epoch=9-step=3070.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at BERT_transfer_learning_ckpts/lightning_logs/version_1/checkpoints/epoch=9-step=3070.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   total_test_accuracy      0.5867741703987122
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'total_test_accuracy': 0.5867741703987122}]